In [39]:
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import shape

In [3]:
# Charge le shapefile
gdf = gpd.read_file("data/RPG_2023/PARCELLES_GRAPHIQUES.gpkg")

In [4]:
gdf["SURF_PARC"].median()

np.float32(1.24)

In [5]:
# Charger le fichier Excel 
correspondance = pd.read_excel("data/correspondances.xlsx", sheet_name="1")

# 'code_culture' et 'nom_culture'
correspondance.columns = correspondance.columns.str.strip()  # nettoyage des noms de colonnes

# Ajouter la colonne 'NOM_CULTURE' 
gdf = gdf.merge(correspondance, how="left", left_on="CODE_CULTU", right_on="code_culture")

gdf = gdf.rename(columns={"nom_culture": "NOM_CULTURE"})

gdf

,ID_PARCEL,SURF_PARC,CODE_CULTU,CODE_GROUP,CULTURE_D1,CULTURE_D2,geometry,code_culture,NOM_CULTURE
0,1,5.01,ORH,3,,,"POLYGON ((620859.121 7049174.538, 621155.742 7...",ORH,Orge d’hiver
1,2,0.43,TRN,6,,,"POLYGON ((518271.056 6380847.362, 518271.088 6...",TRN,Tournesol
2,3,2.34,BTH,1,,,"POLYGON ((653603.846 6971494.032, 653548.732 6...",BTH,Blé tendre d’hiver
3,4,2.31,VRC,21,,,"POLYGON ((450066.555 6519505.157, 449911.365 6...",VRC,Vigne (sauf vigne rouge)
4,5,0.14,SNE,28,,,"POLYGON ((542585.535 6905398.389, 542579.8 690...",SNE,Surface agricole temporairement non admissible...
...,...,...,...,...,...,...,...,...,...
9797400,9866940,0.06,SPH,17,,,"POLYGON ((941769.023 6407193.483, 941769.35 64...",SPH,Prairie avec herbe prédominante et ressources ...
9797401,9866941,0.17,BOR,28,,,"POLYGON ((749684.059 6617162.615, 749683.959 6...",BOR,Bordure de champ
9797402,9866942,3.78,MIS,2,,,"POLYGON ((640902.575 6310402.873, 640902.561 6...",MIS,Maïs (hors maïs doux)
9797403,9866943,3.44,MIS,2,,,"POLYGON ((889837.386 6687978.699, 889853.479 6...",MIS,Maïs (hors maïs doux)


In [6]:
gdf['NOM_CULTURE'].value_counts()


NOM_CULTURE
Prairie de 6 ans et plus (couvert herbacé)                                                                           3043081
Blé tendre d’hiver                                                                                                    885083
Maïs (hors maïs doux)                                                                                                 710194
Prairie temporaire de 5 ans ou moins et autre mélange avec graminées                                                  703112
Jachère (terre arable)                                                                                                572050
                                                                                                                      ...   
Autre culture pérenne et jachère dans les bananeraies                                                                    103
Cultures conduites en inter-rangs (bandes de cultures différentes) – 3 cultures représentant chacune plus de 25 %

In [7]:
gdf_filtered = gdf[(gdf['SURF_PARC'] > 2)]


counts = gdf_filtered['CODE_CULTU'].value_counts()
valid_classes = counts[counts > 15000].index
gdf_filtered = gdf_filtered[gdf_filtered['CODE_CULTU'].isin(valid_classes)]


print(gdf_filtered['NOM_CULTURE'].value_counts(), len(gdf_filtered['CODE_CULTU'].value_counts()), sum(gdf_filtered['CODE_CULTU'].value_counts()))

NOM_CULTURE
Prairie de 6 ans et plus (couvert herbacé)                                              1053420
Blé tendre d’hiver                                                                       609231
Maïs (hors maïs doux)                                                                    395267
Prairie temporaire de 5 ans ou moins et autre mélange avec graminées                     244777
Orge d’hiver                                                                             182955
Colza d’hiver                                                                            164131
Tournesol                                                                                110096
Prairie avec herbe prédominante et ressources fourragères ligneuses présentes             81319
Vigne (sauf vigne rouge)                                                                  74223
Luzerne                                                                                   71774
Triticale d’hiver           

In [20]:
n = 100 # nombre par culture
sample = (
    gdf_filtered.groupby("CODE_CULTU", group_keys=False)
    .apply(lambda x: x.sample(min(len(x), n), random_state=42))
)

In [21]:
sample["SURF_PARC"].sum()

np.float32(13976.4795)

In [22]:
sample.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 2100 entries, 6747702 to 6162497
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   ID_PARCEL     2100 non-null   object  
 1   SURF_PARC     2100 non-null   float32 
 2   CODE_CULTU    2100 non-null   object  
 3   CODE_GROUP    2100 non-null   object  
 4   CULTURE_D1    2100 non-null   object  
 5   CULTURE_D2    2100 non-null   object  
 6   geometry      2100 non-null   geometry
 7   code_culture  2100 non-null   object  
 8   NOM_CULTURE   2100 non-null   object  
dtypes: float32(1), geometry(1), object(7)
memory usage: 155.9+ KB


In [24]:
#sampled = sample.head(1)
sampled = sample
sampled

,ID_PARCEL,SURF_PARC,CODE_CULTU,CODE_GROUP,CULTURE_D1,CULTURE_D2,geometry,code_culture,NOM_CULTURE
6747702,6795423,21.370001,BDH,4,,,"POLYGON ((517492.146 6321037.224, 517497.528 6...",BDH,Blé dur d’hiver
6601093,6647802,3.900000,BDH,4,,,"POLYGON ((390217.291 6539527.808, 390235.211 6...",BDH,Blé dur d’hiver
1900563,1914059,5.520000,BDH,4,,,"POLYGON ((847493.143 6411449.135, 847513.353 6...",BDH,Blé dur d’hiver
695096,699964,4.260000,BDH,4,,,"POLYGON ((812313.277 6303169.877, 812496.498 6...",BDH,Blé dur d’hiver
1892395,1905827,5.440000,BDH,4,,,"POLYGON ((606116.001 6787852.367, 606024.297 6...",BDH,Blé dur d’hiver
...,...,...,...,...,...,...,...,...,...
8005804,8062578,3.800000,VRC,21,,,"POLYGON ((459639.853 6519201.747, 459591.411 6...",VRC,Vigne (sauf vigne rouge)
456078,459234,4.770000,VRC,21,,,"POLYGON ((703163.249 6241133.536, 703206.177 6...",VRC,Vigne (sauf vigne rouge)
6619078,6665904,2.160000,VRC,21,,,"POLYGON ((824067.954 6337123.027, 824029.563 6...",VRC,Vigne (sauf vigne rouge)
3020647,3042041,3.830000,VRC,21,,,"POLYGON ((436139.449 6491680.187, 436227.946 6...",VRC,Vigne (sauf vigne rouge)


In [12]:
import ee 
import geemap

/Users/placiermoise/Documents/remote sensing crop classification/.venv/lib/python3.12/site-packages/geemap/conversion.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [13]:
ee.Authenticate()

True

In [14]:
ee.Initialize()

In [26]:
# reprojettection si nécessaire
if sampled.crs is None or sampled.crs.to_epsg() != 4326:
    sampled = sampled.to_crs(epsg=4326)
    
# Convertir en GeoJSON pour l'affichage
geojson = sampled.__geo_interface__

# Centrer la carte sur les parcelles
center = [sampled.geometry.centroid.y.mean(), sampled.geometry.centroid.x.mean()]

# Créer une carte interactive
Map = geemap.Map(center=center, zoom=5)

# Ajouter les polygones du shapefile
Map.add_geojson(geojson, layer_name="Parcelles")

# Afficher la carte


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
Map

Map(center=[np.float64(46.694319012861136), np.float64(2.1802838700463196)], controls=(WidgetControl(options=[…

In [57]:
#### selectionner un polygone fait à la mano sur Map

polygone_hand_draw = Map.draw_features  

first_polygone = polygone_hand_draw[1] 

geom_hand_poly = first_polygone.geometry()

geom_hand_poly

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "Feature.geometry",
    "arguments": {
      "feature": {
        "functionInvocationValue": {
          "functionName": "Feature",
          "arguments": {
            "geometry": {
              "functionInvocationValue": {
                "functionName": "GeometryConstructors.Polygon",
                "arguments": {
                  "coordinates": {
                    "constantValue": [
                      [
                        [
                          0.933838,
                          49.032466
                        ],
                        [
                          0.933838,
                          49.183499
                        ],
                        [
                          1.31012,
                          49.183499
                        ],
                        [
                          1.31012,
                          49.032466
                        ],
                        [
                          0.933838,
                          49.032466
                        ]
                      ]
                    ]
                  },
                  "geodesic": {
                    "constantValue": false
                  }
                }
              }
            }
          }
        }
      }
    }
  }
})

In [55]:
# Convertir en GEE FeatureCollection
fc = geemap.geopandas_to_ee(sampled, geodesic=False)
fc_geom = fc.geometry()

In [58]:

Geometry_data_collect = geom_hand_poly


# Add a non-zero error margin (e.g., 1 meter)
area_m2 = geom_hand_poly.area(maxError=1)

# Get the area value
print("Area (m²):", area_m2.getInfo())






Area (m²): 459997665.3091503


In [ ]:
start_date = '2023-01-01'
end_date = '2023-12-31'

# Créer une ImageCollection NDVI Sentinel-2
s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterBounds(Geometry_data_collect) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 10)) \
    .map(lambda img: img.clip(fc.geometry()))  # Clip aux parcelles
    

In [59]:
# Créer une ImageCollection Sentinel-2 (niveau 2A = atmosphériquement corrigée avec Sen2Cor) La correction atmosphérique — réalisée par Sen2Cor dans les produits Sentinel-2 Level-2A — transforme les images satellites pour qu'elles reflètent la vraie réflectance du sol, en supprimant les effets de l’atmosphère.
s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterBounds(fc.geometry()) \
    .filterDate(start_date, end_date) \
    .map(lambda img: img.clip(fc.geometry())) \
    .select(['B2', 'B3', 'B4', 'B8'])  # B2: Blue, B3: Green, B4: Red, B8: NIR (tous à 10 m)

# Pas de filtrage par CLOUDY_PIXEL_PERCENTAGE (comme dans le papier)
# Pas besoin de resample ici si on utilise déjà des bandes à 10 m

# Tu peux aussi ajouter un tri temporel :
s2 = s2.sort('system:time_start')

In [ ]:
s2


In [62]:
# Exemple : prendre la médiane temporelle de la collection pour créer une image composite
s2_firts = s2.first()

# Définir les paramètres d'affichage (Red: B4, Green: B3, Blue: B2)
vis_params = {
    'bands': ['B4', 'B3', 'B2'],  # RGB
    'min': 0,
    'max': 3000,
    'gamma': 1.3
}

# Ajouter à la carte
Map.centerObject(fc, zoom=10)  # centrer sur ta zone d’intérêt (fc)
Map.addLayer(s2_firts, vis_params, 'Sentinel-2 RGB')
Map

Map(bottom=45146.0, center=[46.67869751406063, 2.6859889696295496], controls=(WidgetControl(options=['position…

In [63]:
s2_firts.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32632',
   'crs_transform': [10, 0, 300000, 0, -10, 4900020]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32632',
   'crs_transform': [10, 0, 300000, 0, -10, 4900020]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32632',
   'crs_transform': [10, 0, 300000, 0, -10, 4900020]},
  {'id': 'B8',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32632',
   'crs_transform': [10, 0, 300000, 0, -10, 4900020]}],
 'version': 1751376287704355,
 'id': 'COPERNICUS/S2_SR_HARMONIZED/20230101T102339_20

In [6]:
from sklearn.model_selection import train_test_split



gdf_train, gdf_test = train_test_split(
    gdf_filtered,
    test_size=0.2,
    stratify=gdf_filtered['CODE_CULTU'],
    random_state=42
)

print(gdf['CODE_CULTU'].value_counts(normalize=True))
print(gdf_train['CODE_CULTU'].value_counts(normalize=True))
print(gdf_test['CODE_CULTU'].value_counts(normalize=True))